In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

from CLR.train import *
import utils
from CLR.model import CLR

In [2]:
feature_dim = 128
batch_size = 512 
epochs = 10

# CLR train
CLR_lr = 1e-3
CLR_wd = 1e-6
temperature = 0.2

#Classify train
Class_lr = 1e-3

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
CLR_train_data = utils.CIFAR10Pair(root='/datasets/cv_datasets/data', train=True, transform=utils.train_transform, download=True)
CLR_train_loader = DataLoader(CLR_train_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
Class_train_data = utils.CIFAR10(root='/datasets/cv_datasets/data', train=True, transform=utils.test_transform, download=True)
Class_train_loader = DataLoader(Class_train_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_data = utils.CIFAR10(root='/datasets/cv_datasets/data', train=False, transform=utils.test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [5]:
model = CLR(feature_dim).to(device)
CLR_optimizer = optim.Adam(model.parameters(), lr=CLR_lr, weight_decay=CLR_wd)
CLR_loss_fn = utils.NTXentLoss 
Class_optimizer = optim.Adam(model.parameters(), lr=Class_lr)
Class_loss_fn = nn.CrossEntropyLoss()

In [6]:
# Train encoder
for epoch in range(1, epochs + 1):
    train_loss = train_epoch(model, CLR_train_loader, CLR_optimizer, CLR_loss_fn, temperature, device)
    print('Epoch: {}, Loss: {}'.format(epoch, train_loss))

Training:   3%|▎         | 3/97 [00:20<10:56,  6.99s/it]


KeyboardInterrupt: 

In [ ]:
for param in model.encoder.parameters():
    param.requires_grad = False
    
for epoch in range(1, epochs + 1):
    class_loss, class_acc = train_classifier(model, Class_train_loader, Class_optimizer, Class_loss_fn, device)
    print('Classifier - Epoch: {}, Loss: {}, Accuracy: {}'.format(epoch, class_loss, class_acc))
    test_loss, test_acc = test_epoch(model, test_loader, Class_loss_fn, device)
    print('Test - Epoch: {}, Loss: {}, Accuracy: {}'.format(epoch, test_loss, test_acc))